## Process Synthetic Data with Aurora

This notebook shows how to process MTH5 data from a synthetic dataset.

Steps
1. Create the synthetic mth5
2. Get a Run Summary from the mth5
3. Select the station to process and optionally the remote reference station
4. Create a processing config
5. Generate TFs
6. Archive the TFs (in emtf_xml or z-file)

### Here are the modules we will need to import 

In [1]:
import pathlib
import warnings

from aurora.config.config_creator import ConfigCreator
from aurora.pipelines.process_mth5 import process_mth5
from aurora.pipelines.run_summary import RunSummary
from aurora.test_utils.synthetic.make_mth5_from_asc import create_test12rr_h5
from aurora.test_utils.synthetic.paths import DATA_PATH
from aurora.transfer_function.kernel_dataset import KernelDataset

warnings.filterwarnings('ignore')

## Define target folder and mth5 path

By default, the synthetic mth5 file is used for testing in `aurora/tests/synthetic/` and probably already exists on your system if you have run the tests. In the code below, we check if the file exists already, and if not we make it.

**NOTE:** If using a read-only HPC installation, you may not be able to write to the directory where aurora is installed.  In that case, defining your target path as somewhere you have write permission.  In that case, uncommment the READ ONLY INSTALLATION block below.

In [2]:
target_folder = DATA_PATH

## READ ONLY INSTALLATION
# home = pathlib.Path.home()
# target_folder = home.joinpath("aurora_test_folder")
# target_folder.mkdir(parents=True, exist_ok=True)

mth5_path = target_folder.joinpath("test12rr.h5")

If the mth5 doesn't already exist, or you want to re-make it, call `create_test12rr_h5()`

In [3]:
if not mth5_path.exists():
    create_test12rr_h5(target_folder=target_folder)   

## Get a Run Summary

Note that we didn't need to explicitly open the mth5 to do that, we can pass the path if we want.
Run summary takes a list of mth5 paths as input argument.

In [4]:
mth5_run_summary = RunSummary()
mth5_run_summary.from_mth5s([mth5_path,])
run_summary = mth5_run_summary.clone()
run_summary.mini_summary

2023-12-08T10:39:43.924954-0800 | INFO | mth5.mth5 | close_mth5 | Flushing and closing /home/kkappler/software/irismt/aurora/data/test12rr.h5


,survey,station_id,run_id,start,end
0,none,test1,001,1980-01-01 00:00:00+00:00,1980-01-01 11:06:39+00:00
1,none,test2,001,1980-01-01 00:00:00+00:00,1980-01-01 11:06:39+00:00


## Define a Kernel Dataset


In [5]:
kernel_dataset = KernelDataset()
kernel_dataset.from_run_summary(run_summary, "test1", "test2")
kernel_dataset.mini_summary

,survey,station_id,run_id,start,end,duration
0,none,test1,001,1980-01-01 00:00:00+00:00,1980-01-01 11:06:39+00:00,39999.0
1,none,test2,001,1980-01-01 00:00:00+00:00,1980-01-01 11:06:39+00:00,39999.0


## Now define the processing Configuration

The only things we need to provide are our band processing scheme, and the data sample rate to generate a default processing configuration.

The config will get its information about the specific stations to process via the kernel dataset.

**NOTE:** When doing only single station processing you need to specify RME processing (rather than remote reference processing which expects extra time series from another station)

In [6]:
cc = ConfigCreator()
config = cc.create_from_kernel_dataset(kernel_dataset)

# you can export the config to a json by uncommenting the following line
# cfg_json = config.to_json()

2023-12-08T10:39:43.965481-0800 | INFO | aurora.config.config_creator | determine_band_specification_style | Bands not defined; setting to EMTF BANDS_DEFAULT_FILE


## Take a look at the processing configuration

In [7]:
config

{
    "processing": {
        "band_setup_file": "/home/kkappler/software/irismt/aurora/aurora/config/emtf_band_setup/bs_test.cfg",
        "band_specification_style": "EMTF",
        "channel_nomenclature.ex": "ex",
        "channel_nomenclature.ey": "ey",
        "channel_nomenclature.hx": "hx",
        "channel_nomenclature.hy": "hy",
        "channel_nomenclature.hz": "hz",
        "decimations": [
            {
                "decimation_level": {
                    "anti_alias_filter": "default",
                    "bands": [
                        {
                            "band": {
                                "center_averaging_type": "geometric",
                                "closed": "left",
                                "decimation_level": 0,
                                "frequency_max": 0.23828125,
                                "frequency_min": 0.19140625,
                                "index_max": 30,
                                "index_min": 25
 

## Call process_mth5

In [ ]:
show_plot = True
tf_cls = process_mth5(config,
                    kernel_dataset,
                    units="MT",
                    show_plot=show_plot,
                    z_file_path=None,
                )

2023-12-08T10:39:44.030720-0800 | INFO | aurora.pipelines.transfer_function_kernel | show_processing_summary | Processing Summary Dataframe:
2023-12-08T10:39:44.037369-0800 | INFO | aurora.pipelines.transfer_function_kernel | show_processing_summary |   survey station_id run_id  valid  remote  duration     fc dec_level  dec_factor  sample_rate  window_duration  num_samples_window  num_samples  num_stft_windows
0   none      test1    001   True   False   39999.0  False         0         1.0     1.000000            128.0                 128      39999.0             416.0
1   none      test1    001   True   False   39999.0  False         1         4.0     0.250000            512.0                 128       9999.0             103.0
2   none      test1    001   True   False   39999.0  False         2         4.0     0.062500           2048.0                 128       2499.0              25.0
3   none      test1    001   True   False   39999.0  False         3         4.0     0.015625       

In [ ]:
xml_file_base = f"synthetic_test1.xml"
tf_cls.write(fn=xml_file_base, file_type="emtfxml")


In [ ]:
edi_file_base = f"synthetic_test1.edi"
tf_cls.write(fn=edi_file_base, file_type="edi")
